In [1]:
import socketserver
import socket
import struct
import json

import numpy as np


class IPCError(Exception):
    pass

class UnknownMessageClass(IPCError):
    pass

class InvalidSerialization(IPCError):
    pass

class ConnectionClosed(IPCError):
    pass


def _read_objects(sock):
    header = sock.recv(4)
    if len(header) == 0:
        raise ConnectionClosed()
    size = struct.unpack('!i', header)[0]
    data = sock.recv(size - 4)
    if len(data) == 0:
        raise ConnectionClosed()
    return Message.deserialize(json.loads(data))


def _write_objects(sock, objects):
    data = json.dumps([o.serialize() for o in objects])
    sock.sendall(struct.pack('!i', len(data) + 4))
    sock.sendall(data)

def _recursive_subclasses(cls):
    classmap = {}
    for subcls in cls.__subclasses__():
        classmap[subcls.__name__] = subcls
        classmap.update(_recursive_subclasses(subcls))
    return classmap


class Message(object):
    @classmethod
    def deserialize(cls, objects):
        classmap = _recursive_subclasses(cls)
        serialized = []
        for obj in objects:
            if isinstance(obj, Message):
                serialized.append(obj)
            else:
                try:
                    serialized.append(classmap[obj['class']](*obj['args'], **obj['kwargs']))
                except KeyError as e:
                    raise UnknownMessageClass(e)
                except TypeError as e:
                    raise InvalidSerialization(e)
        return serialized

    def serialize(self):
        args, kwargs = self._get_args()
        return {'class': type(self).__name__, 'args': args, 'kwargs': kwargs}

    def _get_args(self):
        return [], {}

    def __repr__(self):
        r = self.serialize()
        args = ', '.join([repr(arg) for arg in r['args']])
        kwargs = ''.join([', {}={}'.format(k, repr(v)) for k, v in r['kwargs'].items()])
        name = r['class']
        return '{}({}{})'.format(name, args, kwargs)


class Client(object):
    def __init__(self, server_address):
        self.addr = server_address
        if isinstance(self.addr, str):
            address_family = socket.AF_UNIX
        else:
            address_family = socket.AF_INET
        self.sock = socket.socket(address_family, socket.SOCK_STREAM)

    def connect(self):
        self.sock.connect(self.addr)

    def close(self):
        self.sock.close()

    def __enter__(self):
        self.connect()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()

    def send(self, objects):
        _write_objects(self.sock, objects)
        return _read_objects(self.sock)

    def recv(self, data):
        print("ooh got data")


class Server(socketserver.ThreadingTCPServer):
    def __init__(self, server_address, callback, bind_and_activate=True):
        if not callable(callback):
            callback = lambda x: []

        class IPCHandler(socketserver.BaseRequestHandler):
            def handle(self):
                while True:
                    try:
                        results = _read_objects(self.request)
                    except ConnectionClosed as e:
                        return
                    _write_objects(self.request, callback(results))

        if isinstance(server_address, str):
            self.address_family = socket.AF_UNIX
        else:
            self.address_family = socket.AF_INET

        socketserver.TCPServer.__init__(self, server_address, IPCHandler, bind_and_activate)

In [2]:
class Event(Message):
    def __init__(self, event_type, **properties):
        self.type = event_type
        self.properties = properties

    def _get_args(self):
        return [self.type], self.properties


class Response(Message):
    def __init__(self, text):
        self.text = text

    def _get_args(self):
        return [self.text], {}


def server_process_request(objects):
    response = [Response('Received {} objects'.format(len(objects)))]
    print('Received objects: {}'.format(objects))
    print('Sent objects: {}'.format(response))
    return response

server_address = ('localhost', 11586)

c = Client(server_address)

In [3]:
def get_samples(note):
    volume = 0.5     # range [0.0, 1.0]
    fs = 48000       # sampling rate, Hz, must be integer
    duration = 0.01#1.0   # in seconds, may be float
    f = note        # sine frequency, Hz, may be float

    # generate samples, note conversion to float32 array
    samples = (np.sin(2*np.pi*np.arange(fs*duration)*f/fs)).astype(np.float32)
    samples = samples + 1
    
    return samples

In [15]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('localhost', 11586))
d = sock.recv(12)
print(d[-4:])

b'EHLO'


In [16]:
# determined that one frame in C++ transmits 11528 bytes
# 8 is header
# b = b''
# while True:
#     b += sock.recv(1)
#     print(len(b))

In [17]:
b = b''
while True:
    size = 11528 # 480(samps)*2(chans)*12(bytes/samp) + 8 (header)
    b += sock.recv(size)
    if len(b) < size:
        continue
    b = b[8:] # first 8 is header
    data = np.frombuffer(b, dtype=np.float32)
    data2 = data*1.1
    data2 = np.clip(data2, 0.00001, 0.99)
    #data2 = np.random.random(480)
    #data2 = get_samples(random.randint(400,600))
    msg2 = data2.tobytes()
    msg = (15).to_bytes(length=4, byteorder='little')
    bint = len(msg2)
    msg += bint.to_bytes(length=4, byteorder='little')
    msg += msg2
    sock.send(msg)
    b = b''

KeyboardInterrupt: 